## Extract basic patient information (adult patients with ICU stay longer than 24 hours)

In [1]:
import numpy as np
import dill
import pandas as pd

import datetime as dt
from tqdm import tqdm
from collections import defaultdict, Counter

import warnings
warnings.filterwarnings("ignore")

In [28]:
def MIMICiii(ADMISSIONS,ICUSTAYS,PATIENTS):
    ADMISSIONS = ADMISSIONS[['SUBJECT_ID', 'HADM_ID','ADMITTIME', 'DISCHTIME','DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION','RACE','HAS_CHARTEVENTS_DATA']]
    ICUSTAYS = ICUSTAYS[['HADM_ID', 'ICUSTAY_ID','DBSOURCE','FIRST_CAREUNIT', 'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID','INTIME', 'OUTTIME', 'LOS']]
    PATIENTS = PATIENTS[['SUBJECT_ID', 'GENDER', 'DOB', 'DOD']]

    # Step 1: Merge DataFrames
    df = pd.merge(PATIENTS, ADMISSIONS, on='SUBJECT_ID', how='left')
    df = pd.merge(df, ICUSTAYS, on='HADM_ID', how='left')
    print(df.shape)
    
    # Step 2: Convert date columns to datetime format
    date_columns = ['DOB', 'ADMITTIME', 'DISCHTIME', 'INTIME', 'OUTTIME','DOD']
    df[date_columns] = df[date_columns].apply(pd.to_datetime)
    
    # Step 3: Calculate AGE
    df['AGE'] = ((df['ADMITTIME'].dt.date - df['DOB'].dt.date) / 365.242).dt.days
    df['AGE'] = df['AGE'].apply(lambda x: 91 if x > 89 else x)
    
    # Step 4: Filter based on AGE, HAS_CHARTEVENTS_DATA
    df = df[df['AGE'] >= 18]
    df = df[df['HAS_CHARTEVENTS_DATA'] == 1]
    print('AGE >= 18',df.shape)
    
    # Step 5: Select relevant columns
    columns_to_keep = [
        'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'GENDER', 'ADMITTIME', 'INTIME', 'DISCHTIME', 'OUTTIME',
        'DOB', 'DOD', 'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'RACE', 'HAS_CHARTEVENTS_DATA',
        'DBSOURCE', 'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID', 'LOS', 'AGE'
    ]
    df = df[columns_to_keep]
    
    # Step 6: Sort by SUBJECT_ID and INTIME, and drop rows with missing INTIME
    df = df.sort_values(by=['SUBJECT_ID', 'INTIME'])
    df = df.dropna(subset=['INTIME'])
    
    # Step 7: Create the FIRST_HADM column (mark the first ADMITTIME for each SUBJECT_ID)
    df['FIRST_HADM'] = df.groupby('SUBJECT_ID')['ADMITTIME'].transform(lambda x: x == x.min()).astype(int)
    
    # Step 8: Create the FIRST_ICU column (mark the first INTIME for each HADM_ID within SUBJECT_ID)
    df['FIRST_ICU'] = df.groupby(['SUBJECT_ID', 'HADM_ID'])['INTIME'].transform(lambda x: x == x.min()).astype(int)
    
    # Step 9: Filter to keep only rows where both FIRST_HADM and FIRST_ICU are 1
    df = df[(df['FIRST_HADM'] == 1) & (df['FIRST_ICU'] == 1)]
    print('FIRST_HADM, FIRST_ICU',df.shape)
    
    # Step 10: Filter for LOS greater than 1
    df = df[df['LOS'] > 1]
    print('LOS',df.shape)

    # Drop rows with missing critical columns
    df = df.dropna(subset=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ADMITTIME', 'INTIME', 'DOB'])
    print(df.shape)
    
    df['DIEINHOSPITAL'] = df.apply(
    lambda row: 1 if row['DOD'] >= row['ADMITTIME'] and row['DOD'] <= row['DISCHTIME'] else 0,
    axis=1)
    
    print(df[['DOD','ADMITTIME','DISCHTIME']].head(1))

    print('DIEINHOSPITAL:',df['DIEINHOSPITAL'].value_counts())

    return df

In [30]:
PATIENTS = pd.read_csv('input/PATIENTS.csv.gz')
ADMISSIONS = pd.read_csv('input/ADMISSIONS.csv.gz')
ICUSTAYS = pd.read_csv('input/ICUSTAYS.csv.gz')

ADMISSIONS.columns = ADMISSIONS.columns.str.upper()
ICUSTAYS.columns = ICUSTAYS.columns.str.upper()
PATIENTS.columns = PATIENTS.columns.str.upper()

ADMISSIONS = ADMISSIONS.rename(columns={'ETHNICITY':'RACE'})
ICUSTAYS = ICUSTAYS.rename(columns={'STAY_ID':'ICUSTAY_ID'})

df = MIMICiii(ADMISSIONS,ICUSTAYS,PATIENTS)
df.shape

(62722, 21)
AGE >= 18 (52916, 22)
FIRST_HADM, FIRST_ICU (38470, 24)
LOS (32557, 24)
(32557, 24)
           DOD           ADMITTIME           DISCHTIME
323 2102-06-14 2101-10-20 19:08:00 2101-10-31 13:58:00
DIEINHOSPITAL: 0    28971
1     3586
Name: DIEINHOSPITAL, dtype: int64


(32557, 25)

In [31]:
df.to_csv('output/III_patients.csv',index=False)

In [26]:
def MIMICiv(ADMISSIONS,ICUSTAYS,PATIENTS):
    # Select relevant columns for ADMISSIONS, ICUSTAYS, and PATIENTS
    ADMISSIONS = ADMISSIONS[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'RACE']]
    ICUSTAYS = ICUSTAYS[['HADM_ID', 'ICUSTAY_ID', 'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'INTIME', 'OUTTIME', 'LOS']]
    PATIENTS['DOB'] = PATIENTS['ANCHOR_YEAR'] - PATIENTS['ANCHOR_AGE']
    PATIENTS = PATIENTS[~((PATIENTS.ANCHOR_YEAR_GROUP == '2008 - 2010') | (PATIENTS.ANCHOR_YEAR_GROUP == '2011 - 2013'))]
    PATIENTS = PATIENTS[['SUBJECT_ID', 'GENDER', 'DOB', 'DOD','ANCHOR_YEAR_GROUP']]
    
    # Merge the datasets
    df = pd.merge(PATIENTS, ADMISSIONS, on='SUBJECT_ID', how='left')
    df = pd.merge(df, ICUSTAYS, on='HADM_ID', how='left')
    print(df.shape,'\n',df.ANCHOR_YEAR_GROUP.value_counts())
    
    # Convert date columns to datetime
    date_columns = ['ADMITTIME', 'DISCHTIME', 'INTIME', 'OUTTIME','DOD']
    df[date_columns] = df[date_columns].apply(pd.to_datetime)
    
    # Create 'DOB' as a complete date by assuming January 1st for each year
    df['DOB'] = pd.to_datetime(df['DOB'].astype(str) + '-01-01')
    
    # Calculate 'AGE' based on 'ADMITTIME' and 'DOB', and adjust for ages above 89
    df['AGE'] = ((df['ADMITTIME'].dt.date - df['DOB'].dt.date) / 365.242).dt.days
    df['AGE'] = df['AGE'].apply(lambda x: 91 if x > 89 else x)
    
    # Filter out patients under 18
    df = df[df['AGE'] >= 18]
    print('AGE >= 18',df.shape)
    
    # Select specific columns for the final dataframe
    df = df[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ADMITTIME', 'INTIME', 'DISCHTIME', 'OUTTIME', 
             'GENDER', 'DOB', 'DOD', 'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'RACE', 
             'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'LOS', 'AGE']]
    
    # Sort the dataframe by 'SUBJECT_ID' and 'INTIME'
    df = df.sort_values(by=['SUBJECT_ID', 'INTIME'])
    
    # Drop rows where 'INTIME' is missing
    df = df.dropna(subset=['INTIME'])
    
    # Create a flag for the first admission for each patient
    df['FIRST_HADM'] = df.groupby('SUBJECT_ID')['ADMITTIME'].transform(lambda x: x == x.min()).astype(int)
    
    # Create a flag for the first ICU stay for each patient-HADM combination
    df['FIRST_ICU'] = df.groupby(['SUBJECT_ID', 'HADM_ID'])['INTIME'].transform(lambda x: x == x.min()).astype(int)
    
    # Filter for rows where both FIRST_HADM and FIRST_ICU are 1
    df = df[(df['FIRST_HADM'] == 1) & (df['FIRST_ICU'] == 1)]
    print('FIRST_HADM, FIRST_ICU',df.shape)
    
    # Keep rows with LOS greater than 1
    df = df[df['LOS'] > 1]
    print('LOS',df.shape)
    
    # Drop rows with missing critical columns
    df = df.dropna(subset=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ADMITTIME', 'INTIME', 'DOB'])

    df['ADMITTIME'] = df['ADMITTIME'].dt.date
    df['DISCHTIME'] = df['DISCHTIME'].dt.date
    
    df['DIEINHOSPITAL'] = ((df['ADMITTIME'] <= df['DOD']) & (df['DOD'] <= df['DISCHTIME'])).astype(int)
    print(df[['DOD','ADMITTIME','DISCHTIME']].head(1))

    print('DIEINHOSPITAL:',df['DIEINHOSPITAL'].value_counts())

    return df

In [32]:
PATIENTS = pd.read_csv('input/PATIENTS.csv.gz')
ADMISSIONS = pd.read_csv('input/ADMISSIONS.csv.gz')
ICUSTAYS = pd.read_csv('input/ICUSTAYS.csv.gz')

ADMISSIONS.columns = ADMISSIONS.columns.str.upper()
ICUSTAYS.columns = ICUSTAYS.columns.str.upper()
PATIENTS.columns = PATIENTS.columns.str.upper()

ADMISSIONS = ADMISSIONS.rename(columns={'ETHNICITY':'RACE'})
ICUSTAYS = ICUSTAYS.rename(columns={'STAY_ID':'ICUSTAY_ID'})

df = MIMICiv(ADMISSIONS,ICUSTAYS,PATIENTS)
df.shape

(285297, 18) 
 2014 - 2016    121728
2017 - 2019     99377
2020 - 2022     64192
Name: ANCHOR_YEAR_GROUP, dtype: int64
AGE >= 18 (208036, 19)
FIRST_HADM, FIRST_ICU (34079, 20)
LOS (27636, 20)
          DOD   ADMITTIME   DISCHTIME
47 2162-12-11  2162-02-17  2162-03-04
DIEINHOSPITAL: 0    24453
1     3183
Name: DIEINHOSPITAL, dtype: int64


(27636, 21)

In [33]:
df.to_csv('output/IV_patients.csv',index=False)